In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

pd.set_option('display.max_columns', 25)

In [9]:
df_10_with_covid = pd.read_csv(r'C:\Users\Fib0nacci\Desktop\PublicAssistanceFundedProjectsDetails.csv')

In [10]:
df_10_with_covid.head()

,disasterNumber,declarationDate,incidentType,pwNumber,applicationTitle,applicantId,damageCategoryCode,dcc,damageCategory,projectSize,county,countyCode,state,stateCode,stateNumberCode,projectAmount,federalShareObligated,totalObligated,obligatedDate,hash,lastRefresh,id
0,1239,1998-08-26T04:00:00.000Z,Severe Storm(s),41,Not Provided,463-99463-00,C - Roads & Bridges,C,Roads & Bridges,Small,Uvalde,463.0,Texas,TX,48,1489.98,1117.49,1158.77,1998-10-23T11:29:35.000Z,70c217e884c4091e9a25ef4fc4a4bbc9,2021-03-19T16:33:13.713Z,5ee7590f556129600dd511bd
1,1239,1998-08-26T04:00:00.000Z,Severe Storm(s),51,Not Provided,463-99463-00,C - Roads & Bridges,C,Roads & Bridges,Small,Uvalde,463.0,Texas,TX,48,11300.00,8475.00,8788.01,1998-10-23T11:29:35.000Z,b12a8d74084468fdf238fc8a97228472,2021-03-19T16:33:13.714Z,5ee7590f556129600dd511be
2,1239,1998-08-26T04:00:00.000Z,Severe Storm(s),43,Not Provided,463-99463-00,C - Roads & Bridges,C,Roads & Bridges,Small,Uvalde,463.0,Texas,TX,48,3486.00,2614.50,2711.06,1998-10-23T11:29:35.000Z,b861bae962edd96f32e543f2bac95cb6,2021-03-19T16:33:13.713Z,5ee7590f556129600dd511bf
3,1239,1998-08-26T04:00:00.000Z,Severe Storm(s),2,(L),465-19792-01,F - Public Utilities,F,Public Utilities,Large,Val Verde,465.0,Texas,TX,48,423791.13,317843.35,333171.72,1998-09-17T16:00:59.000Z,6a222a9d0d175f87d0ba16b59d692e0b,2020-06-15T11:18:39.486Z,5ee7590f556129600dd511c1
4,1239,1998-08-26T04:00:00.000Z,Severe Storm(s),47,Not Provided,463-99463-00,C - Roads & Bridges,C,Roads & Bridges,Small,Uvalde,463.0,Texas,TX,48,3819.67,2864.75,2970.56,1998-10-23T11:29:35.000Z,ebfabdad3c81c74234d056102dd81c15,2021-03-19T16:33:13.714Z,5ee7590f556129600dd511c0


In [11]:
df_10_with_covid.shape

(732340, 22)

In [12]:
df_10_with_covid = df_10_with_covid.drop(['pwNumber', 'applicantId', 'dcc', 'damageCategory', 'county', 
           'countyCode', 'stateNumberCode', 'obligatedDate', 'hash', 'lastRefresh', 'id'], axis=1)
df_10_with_covid.head()
display(df_10_with_covid.shape)

(732340, 11)

In [13]:
df_10_with_covid['declarationDate'] = pd.to_datetime(df_10_with_covid['declarationDate'], format='%Y-%m-%d')
df_10_with_covid.head(1)

,disasterNumber,declarationDate,incidentType,applicationTitle,damageCategoryCode,projectSize,state,stateCode,projectAmount,federalShareObligated,totalObligated
0,1239,1998-08-26 04:00:00+00:00,Severe Storm(s),Not Provided,C - Roads & Bridges,Small,Texas,TX,1489.98,1117.49,1158.77


In [14]:
#this code was adapted from geeksforgeeks
#DO NOT Rerun
#We only want 10 year data, so we took data from the end of 2010 to the end of 2020.
mask = (df_10_with_covid['declarationDate'] > '2010-12-31') & (df_10_with_covid['declarationDate'] <= '2020-12-31')
df_10_with_covid = df_10_with_covid.loc[mask]
df_10_with_covid.head()

,disasterNumber,declarationDate,incidentType,applicationTitle,damageCategoryCode,projectSize,state,stateCode,projectAmount,federalShareObligated,totalObligated
462426,1952,2011-01-26 05:00:00+00:00,Flood,CBMIB01 - BIA Roads - Emergency Protective Mea...,B - Protective Measures,Small,California,CA,3893.00,2919.75,2919.75
462427,1952,2011-01-26 05:00:00+00:00,Flood,LJIRA01 - Debris Removal - 88 Culverts and Veg...,A - Debris Removal,Small,California,CA,45936.76,34452.57,34452.57
462428,1952,2011-01-26 05:00:00+00:00,Flood,PBMIE01 - Flood Waters Damaged Casino,E - Public Buildings,Small,California,CA,3263.00,2447.25,2447.25
462429,1952,2011-01-26 05:00:00+00:00,Flood,RCIR-01 - Seven Sites - Roads and Slope Damage,C - Roads & Bridges,Small,California,CA,10602.25,7951.69,7951.69
462430,1952,2011-01-26 05:00:00+00:00,Flood,SMR25- Old Highland Canal-Debris Removal,A - Debris Removal,Small,California,CA,19214.59,14410.94,14410.94


In [15]:
# Remove any states that were not obligated to any federal money
#DO NOT Rerun
df_10_with_covid = df_10_with_covid.loc[df_10_with_covid['federalShareObligated'] != 0.00].copy()

In [19]:
#We are keeping all covid data in this time.
df_10_with_covid.to_csv('./data/10-year-covid.csv')

In [21]:
df10_covid = pd.read_csv('./data/10-year-covid.csv')
df10_covid.head(2)

,Unnamed: 0,disasterNumber,declarationDate,incidentType,applicationTitle,damageCategoryCode,projectSize,state,stateCode,projectAmount,federalShareObligated,totalObligated
0,462426,1952,2011-01-26 05:00:00+00:00,Flood,CBMIB01 - BIA Roads - Emergency Protective Mea...,B - Protective Measures,Small,California,CA,3893.00,2919.75,2919.75
1,462427,1952,2011-01-26 05:00:00+00:00,Flood,LJIRA01 - Debris Removal - 88 Culverts and Veg...,A - Debris Removal,Small,California,CA,45936.76,34452.57,34452.57


In [22]:
print(df10_covid['totalObligated'].sum())

85616140220.51999
